In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [57]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))   
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [58]:
string_to_int  = {ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [ string_to_int[c] for c in s]
decode = lambda s: ''.join([int_to_string[c] for c in s])

In [59]:
encode('Rahul')

[42, 54, 61, 74, 65]

In [60]:
decode([42, 54, 61, 74, 65])

'Rahul'

In [61]:
data = torch.tensor(encode(text),dtype = torch.long)
print(data[:200])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39,
        30,  1, 39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45,
        43, 44, 42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,
         1, 38, 29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,
         1, 47, 39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39,
        42, 42, 39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38,
        29, 47])


In [62]:
block_size = 8
n=int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]



In [63]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]


for t in range(block_size):
    contex = x[:t+1]
    target = y[t]
    print('when input is', contex,'target is',target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [72]:
batch_size=32
block_size = 128
max_iter=1000
# vocab_size=int(0.8*len(data))
max_iters = 3000
learning_rate = 3e-4
eval_iters = 258
vocab_size= len(chars)
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs')
print(x)
print('target')
print(y)

inputs
tensor([[73,  1, 76,  ..., 54,  1, 59],
        [74, 71, 58,  ...,  1, 62, 73],
        [68, 58, 72,  ..., 73, 61, 54],
        ...,
        [68, 67,  1,  ..., 71, 68, 76],
        [58, 71, 58,  ..., 56, 68, 74],
        [67,  1, 76,  ..., 66,  1, 54]])
target
tensor([[ 1, 76, 54,  ...,  1, 59, 62],
        [71, 58, 64,  ..., 62, 73,  1],
        [58, 72,  1,  ..., 61, 54, 73],
        ...,
        [67,  1, 73,  ..., 68, 76, 57],
        [71, 58,  1,  ..., 68, 74, 67],
        [ 1, 76, 61,  ...,  1, 54,  1]])


In [73]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [65]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        self.vocab_size = vocab_size
            
    def forward(self,index,target=None):
        logits = self.token_embedding_table(index)
        
        if target is None:
            loss = None
            
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = target.view(B*T)
            loss = F.cross_entropy(logits,targets) 
        return logits,loss
    
    def generate(self,index,max_tokens):
        # index is (B,T) array of indices in contex
        for _ in range(max_tokens):
            # get the prediction
            logits,loss = self.forward(index)
            #focus only on last time stamp
            logits =  logits[:,-1,:] #beacome (B, C)
            #apply softmax to get probability
            prob = F.softmax(logits, dim=-1)
            #sample for distrubution minimum=1
            index_next = torch.multinomial(prob,num_samples=1) #(0,1)
            #append sample index to running index
            index = torch.cat((index,index_next),dim=-1)
            
        return index
        
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# context = torch.zeros((1,1),dtype=torch.long,device=device)
# generate_char = decode(m.generate(contex,max_tokens=500)[0].tolist())
# print(generate_char)


In [66]:
        
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# context = torch.zeros((1,3),dtype=torch.long,device=device)

# logist,loss = m.forward(contex)

# logist[:,-1,:]
context = torch.LongTensor([[62, 67, 60, 71, 1, 74]])

logit=m.token_embedding_table(contex)
print(logit.shape)
index=m.generate(context,500)
decode(index[0].tolist())



torch.Size([8, 81])


'ingr u\ufeff[ix70aWt_"HY?;&VN1phN9852hO_x[a1kaaor\'_p))HZ]n;qGE4XJQOQxLgQ vp?Z&cwCCL8)6.nJ:c,p\ufeffYKwK\'3\ufeffge bn*[e5W7g\ufeffM&;1Dkd4kh-t\ufeff0jNAFrPJ6.hNqSx4-4p!p\')!R!3Mi1Fnn17W!fo1HE:hOS1\'ACLxV_Lc0Kwa(5RWnX[*U\ufeff\':M?\'Z.xnYOu&hd9Z]5:46Gh\')oUvi;sGe[*n];rRZq[*3ojZh7JJMe5,65)Jtt;A1m\nd:6qufC591wn?ifK1Luf\'?B3n8Ho&KHLTUTM,C:MVr\'Fvbl,[f96aG?yHQ?Go!Y;4gaYd"k\ncR1B1;:BNA&he;yCXB9uaL[i*nJ9 y]iKg(VIoN;;ALcR!CDyL\ufeff2L&hy\ncm!_iUZ!U4dzovpqcDI027gVO(7K\ufeff.os23CHLx1UdG3Et5[.7.6oTrDG[R1Ag]?_euWe92Q8Jj5vJJQ&q1\')Y*zzxW(c6u1aj:HfY2LHB.CzSTG!yFD2A'

In [67]:
print(torch.__file__)

f:\GenAI\LLM_FROM_SCRATCH\venv\Lib\site-packages\torch\__init__.py


In [78]:


optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate,weight_decay=0.01)


for iter in range(max_iters):
    if  iter % eval_iters==0:
        losses = estimate_loss()
        # print(losses)
        print(f'step: {iter}, train loss: {losses["train"]}, val loss: {losses["val"]}')
    #sample bacth of data
    xb,yb = get_batch('train')
    # print(xb)
    logits = model.token_embedding_table(xb)
    # print(logits.shape)
    #evalute the loss
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step: 0, train loss: 4.852230072021484, val loss: 4.8716278076171875
step: 258, train loss: 4.738840579986572, val loss: 4.754951477050781
step: 516, train loss: 4.6255364418029785, val loss: 4.644659042358398
step: 774, train loss: 4.5163726806640625, val loss: 4.535719871520996
step: 1032, train loss: 4.412199974060059, val loss: 4.432003498077393
step: 1290, train loss: 4.3087639808654785, val loss: 4.3291449546813965
step: 1548, train loss: 4.211060523986816, val loss: 4.23232364654541
step: 1806, train loss: 4.116181373596191, val loss: 4.138118267059326
step: 2064, train loss: 4.025889873504639, val loss: 4.047524929046631
step: 2322, train loss: 3.9367308616638184, val loss: 3.959258556365967
step: 2580, train loss: 3.8526437282562256, val loss: 3.874993324279785
step: 2838, train loss: 3.769577980041504, val loss: 3.7947838306427
3.6897130012512207


In [69]:
print(torch.__version__)

2.2.1+cpu


In [71]:
index = torch.zeros((2,3),dtype=torch.long)

t=decode(m.generate(index,500)[0].tolist())
# decode(t[0].tolist())
t

'\n\n\nfO\ufeff\'aPVqn_Z3q,rgurFChA,Ip6Pe8O&;sAdAVY?zX-fn_M8X::Jf9]ILTm4pG!Z\'rEfRdWyU*o38H? V!LWos26jZL4-.Z(TVzR1UZ:4?7I9Ifn\ufeffq3lW"6A VI \ufeff\'9\ufeffQ!Z X0**os8vJn73ZQ!B*gV!0G7rur_i;&o!3!0e2!BlyHXScmo13x0SAyfu8nMb)LHOrp*YVY(T_Z;;RyFX56G\nb7YB*KJZQnq5ei"\n38\ufeff[*U8nv(fcQf21C xt]DP6JTmxWo(_2P\ny:BOsd&B&*s)6qa6\'J[f;iuo\'S"YKwKODQ8R1(aS\nY?Q-HZ_FKH84-7_"jcw3,l7W_9En&\'?pTvAvI Lfz]\nv*3cDgNVV9q)WPTvOG?u\nf;EqEDp1;uXg4p4C.O_zVO\')Jd0\njYX\nm6L,fHO\ufeffl&t:zLa x8sp 4-t\ufeffDZI7J6GckwR1)Mz;16jhy\ne[3em?DLq9]B1gFZDk2:NPJdX!NAd)WKmOx,9.ohOJp_Zqur\':'